# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [14]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [21]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv(r"C:\Users\arzan\OneDrive\Desktop\cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
1,1,farsund,58.0948,6.8047,13.30,100,0,7.65,NO,1666108228
2,2,new norfolk,-42.7826,147.0587,11.72,58,12,1.34,AU,1666108230
3,3,jamestown,42.0970,-79.2353,5.77,77,100,9.77,US,1666107934
4,4,lanzhou,36.0564,103.7922,14.53,48,59,1.20,CN,1666108230


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [22]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
"Lng",
"Lat",
geo = True,
tiles = "OSM",
frame_width = 600,
frame_height = 400,
size = "Humidity",
scale = 0.5,    
color = "City")

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [23]:
# Narrow down cities that fit criteria
vacation_df = city_data_df.loc[(city_data_df['Max Temp'] > 22) & 
                               (city_data_df['Max Temp'] < 27) & 
                                (city_data_df['Humidity'] > 40) &
                               (city_data_df['Wind Speed'] < 4.5) & 
                               (city_data_df['Cloudiness'] < 30), :]

# Drop any rows with null values
vacation_df.dropna()

# Display sample data
vacation_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
20,20,avarua,-21.2078,-159.7750,22.03,73,20,2.57,CK,1666108220
45,45,kapaa,22.0752,-159.3190,22.99,84,0,3.60,US,1666108257
51,51,hilo,19.7297,-155.0900,26.27,83,0,2.57,US,1666108260
53,53,lasa,46.6166,10.7002,23.49,68,11,2.01,IT,1666108019
54,54,vaini,-21.2000,-175.2000,23.09,100,20,2.57,TO,1666108262
63,63,banda,25.4833,80.3333,24.62,52,0,2.68,IN,1666108268
135,135,ambon,-3.6954,128.1814,26.08,83,20,1.54,ID,1666108315
152,152,kahului,20.8947,-156.4700,23.80,60,0,3.09,US,1666108246
197,197,gat,31.6100,34.7642,24.38,100,0,3.69,IL,1666108356
198,198,buala,-8.1450,159.5921,26.74,82,17,3.00,SB,1666108357


### Step 3: Create a new DataFrame called `hotel_df`.

In [25]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = vacation_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
20,avarua,CK,-21.2078,-159.7750,73,
45,kapaa,US,22.0752,-159.3190,84,
51,hilo,US,19.7297,-155.0900,83,
53,lasa,IT,46.6166,10.7002,68,
54,vaini,TO,-21.2000,-175.2000,100,
63,banda,IN,25.4833,80.3333,52,
135,ambon,ID,-3.6954,128.1814,83,
152,kahului,US,20.8947,-156.4700,60,
197,gat,IL,31.6100,34.7642,100,
198,buala,SB,-8.1450,159.5921,82,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [26]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":1
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Define an API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
avarua - nearest hotel: Paradise Inn
kapaa - nearest hotel: Pono Kai Resort
hilo - nearest hotel: Dolphin Bay Hotel
lasa - nearest hotel: Hotel "Drei Kreuz"
vaini - nearest hotel: Keleti International Resort
banda - nearest hotel: #acnindiafy21
ambon - nearest hotel: Hotel Hero
kahului - nearest hotel: Maui Seaside Hotel
gat - nearest hotel: No hotel found
buala - nearest hotel: No hotel found
tikaitnagar - nearest hotel: No hotel found
salalah - nearest hotel: Muscat International Hotel
kailua - nearest hotel: No hotel found
santa rosalia - nearest hotel: Hotel del Real
quelimane - nearest hotel: Mabassa
narwar - nearest hotel: No hotel found
roebourne - nearest hotel: No hotel found
saint-francois - nearest hotel: Chez Lily
capoterra - nearest hotel: Rosa Hotel
kalemie - nearest hotel: Hotel du Lac
stolac - nearest hotel: Bregava
port-de-bouc - nearest hotel: B&B Hôtel
dzaoudzi - nearest hotel: 2.0 hôtel
harindanga - nearest hotel: No hotel found
lahij - nearest

,City,Country,Lat,Lng,Humidity,Hotel Name
20,avarua,CK,-21.2078,-159.7750,73,Paradise Inn
45,kapaa,US,22.0752,-159.3190,84,Pono Kai Resort
51,hilo,US,19.7297,-155.0900,83,Dolphin Bay Hotel
53,lasa,IT,46.6166,10.7002,68,"Hotel ""Drei Kreuz"""
54,vaini,TO,-21.2000,-175.2000,100,Keleti International Resort
63,banda,IN,25.4833,80.3333,52,#acnindiafy21
135,ambon,ID,-3.6954,128.1814,83,Hotel Hero
152,kahului,US,20.8947,-156.4700,60,Maui Seaside Hotel
197,gat,IL,31.6100,34.7642,100,No hotel found
198,buala,SB,-8.1450,159.5921,82,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [27]:
%%capture --no-display

# Configure the map plot
map_plot_hotels = hotel_df.hvplot.points(
"Lng",
"Lat",
geo = True,
tiles = "OSM",
frame_width = 1000,
frame_height = 800,
color = "City",
hover_cols=["Hotel Name", "Country"])

# Display the map
map_plot_hotels

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)